In [3]:
#!pip install transformers

In [2]:
from transformers import pipeline 
from bs4 import BeautifulSoup # web scraping library
import requests

In [5]:
summarizer = pipeline('summarization')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [4]:
summarizer

In [6]:
URL = 'https://hackernoon.com/what-netflix-should-consider-before-diving-into-gaming-vr2i34c6'

In [7]:
r = requests.get(URL)

In [9]:
r.text

'<!DOCTYPE html><html lang="en"><head><script src="https://kit.fontawesome.com/0dfa7864cb.js" crossorigin="anonymous"></script><meta name="google-site-verification" content="xSsN5mRM5xbHVI00M8uekCnuJ47Da0ER2mYuGhaPM9o"/><link rel="stylesheet" href="/fonts/HackerNoonFont/font-hackernoon.css"/><script>var _iub = _iub || [];\n              _iub.csConfiguration = {"enableCcpa":true,"countryDetection":true,"ccpaAcknowledgeOnDisplay":true,"lang":"en","siteId":1848357,"gdprAppliesGlobally":false,"cookiePolicyId":18778700, "banner":{ "acceptButtonDisplay":true,"customizeButtonDisplay":true,"acceptButtonColor":"#00d500","acceptButtonCaptionColor":"white","customizeButtonColor":"#212121","customizeButtonCaptionColor":"white","rejectButtonColor":"#0073CE","rejectButtonCaptionColor":"white","position":"float-bottom-left","textColor":"white","backgroundColor":"#000001" }};</script><script src="//cdn.iubenda.com/cs/iubenda_cs.js" charSet="UTF-8" async=""></script><meta name="viewport" content="width

In [17]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1','p']) # tags: 'h1' for subtitles and 'p' for paragraphs

In [19]:
text = [result.text for result in results]
ARTICLE = ''.join(text)

In [22]:
#ARTICLE

In [26]:
# "chunk" article into sentences by replacing punctuation marks with end of sentence tags
ARTICLE = ARTICLE.replace('. ', '.<eos>')
ARTICLE = ARTICLE.replace('! ', '!<eos>')
ARTICLE = ARTICLE.replace('? ', '?<eos>')
sentences = ARTICLE.split('<eos>')

In [27]:
sentences[0:5]

['What Netflix Should Consider Before Diving into Gamingcreating demand where there is none & sporadically spamming the world with my opinions The gaming industry is a lucrative space.',
 ' With over 2.',
 '7 billion gamers spending $159.',
 '3 billion USD in 2020, and an estimated annual growth rate of over 9% in the next five years, it’s no surprise big tech is eyeing this market.',
 ' In recent years, tech giants like Microsoft and Apple have been fixated on gaming subscriptions, which if successful could tilt the scales and shift the power from game studios and publishers to a few digital distributors.']

In [31]:
#limit chuncks to 500 words
max_chunk = 500
current_chunk = 0
chunks =[]

for sentence in sentences:
    if len(chunks)==current_chunk+1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk+= 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

0


In [32]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [35]:
len(chunks[2].split(' '))

77

In [36]:
# max_length, min_length: max min words in summary
res = summarizer(chunks, max_length = 120, min_length = 30, do_sample = False)

Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors


In [37]:
res

[{'summary_text': 'the information was the first to break the news (via The Verge) on Netflix’s plans to expand into the gaming sphere . Netflix is looking to hire an executive to supervise new gaming efforts . the company is exploring a bundle of games similar to apple Arcade .'},
 {'summary_text': 'Netflix faces some pretty stiff competition in the gaming subscription market . 207. 6 million Netflix subscribers seems like a large number . but when compared with over 1 billion iPhone users, that no longer seems an advantage .'},
 {'summary_text': 'the rise of the metaverse makes me skeptical about the gaming subscription model in general . create your free account to unlock your custom reading experience . Quality Weekly Reads About Technology Infiltrating Everything .'}]

In [40]:
res[0]['summary_text']

'the information was the first to break the news (via The Verge) on Netflix’s plans to expand into the gaming sphere . Netflix is looking to hire an executive to supervise new gaming efforts . the company is exploring a bundle of games similar to apple Arcade .'

In [41]:
text = ' '.join([sum['summary_text'] for sum in res])

In [42]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)